In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import ltrim,rtrim,trim,col
from pyspark.sql.functions import *
from pyspark import SparkContext
spark = SparkSession.builder.appName("MyFirstCSV").getOrCreate()


In [4]:
#CREATE DATAFRAME FROM CSV FILE
oij_df = spark.read.csv( path="OIJ.csv", sep=",", header=True,quote='"',inferSchema=True,)
inec_df = spark.read.csv( path="INEC.csv", sep=";", header=True,quote='"',inferSchema=True,)

In [5]:
#Function to remove spaces from the beginning and end of the string
def remove_spaces(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito' or col == 'Provincia, cantón y distrito':
            df = df.withColumn(col, trim(col))
    return df

#Function to parser the string to lowercase
def to_lower_case(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito' or col == 'Provincia, cantón y distrito':
            df = df.withColumn(col, lower(col))
    return df


oij_df = remove_spaces(oij_df)
inec_df = remove_spaces(inec_df)

oij_df = to_lower_case(oij_df)
inec_df = to_lower_case(inec_df)

oij_df.show(5)
inec_df.show(5)

22/11/24 19:50:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Delito, SubDelito, Fecha, Hora, Victima, SubVictima, Edad, Genero, Nacionalidad, Provincia, Canton, Distrito, 
 Schema: Delito, SubDelito, Fecha, Hora, Victima, SubVictima, Edad, Genero, Nacionalidad, Provincia, Canton, Distrito, _c12
Expected: _c12 but found: 
CSV file: file:///mnt/c/Users/kevin/desktop/proyecto3bases/ProyectoBases3/OIJ.csv
+------+-----------+-------------------+-------------------+--------+--------------------+-------------+------+------------+----------+----------+--------+----+
|Delito|  SubDelito|              Fecha|               Hora| Victima|          SubVictima|         Edad|Genero|Nacionalidad| Provincia|    Canton|Distrito|_c12|
+------+-----------+-------------------+-------------------+--------+--------------------+-------------+------+------------+----------+----------+--------+----+
|ASALTO|ARMA BLANCA|2021-06-03 00:00:00|00:00:00 - 02:59:59|VEHICULO|SERVICIO PU

In [ ]:
#Function find list the not match values in the dataframes
def find_non_matches(df1,df2):
    non_matches = []
    for col in df1.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito':
            for row in df1.select(col).distinct().collect():
                if not df2.filter(df2['Provincia, cantón y distrito'] == row[col]).collect():
                    non_matches.append(row[col])
    return non_matches

non_matches_oij = find_non_matches(oij_df,inec_df)


#Function to parser the string to lowercase
def to_lower_case(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito' or col == 'Provincia, cantón y distrito':
            df = df.withColumn(col, lower(col))
    return df


oij_df = remove_spaces(oij_df)
inec_df = remove_spaces(inec_df)

oij_df = to_lower_case(oij_df)
inec_df = to_lower_case(inec_df)

nonMatches = find_non_matches(oij_df,inec_df)
print(nonMatches)
#oij_df.show(5)
#inec_df.show(5)


In [ ]:
#Function to replace the accents in column Provincia, cantón y distrito in inec_df
def replace_accents(df):
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'á', 'a'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'é', 'e'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'í', 'i'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'ó', 'o'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'ú', 'u'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'ñ', 'n'))
    return df

inec_df = replace_accents(inec_df)
nonMatches = find_non_matches(oij_df,inec_df)
print(nonMatches)


In [ ]:
#Fucntion to separate column Provincia, cantón y distrito in inec_df in three columns


In [8]:
def generate_new_columns(df):
    columns = ['Provincia', 'Canton', 'Distrito','Tasa neta de participación', 'Porcentaje de población económicamente inactiva', 'Relación de dependencia económica']
    new_df = spark.createDataFrame(data =[], schema = columns)
    Provincia = ''
    Canton = ''
    counter = 0
    for row in df.collect():
        if row[0] == '':
            if counter == 3:
                counter = 0
            else:
             counter += 1
        if counter == 1:
            Provincia = row[0]
        if counter == 2:
            Canton = row[0]
        if counter == 3:
            new_df = new_df.union(spark.createDataFrame([(Provincia, Canton, row[2], row[3], row[4], row[5])], columns))
    return new_df
    
generate_new_columns(inec_df)


ValueError: can not infer schema from empty dataset

In [ ]:
#Function to remove column Provincia, cantón y distrito in inec_df
def remove_column(df):
    df = df.drop('Provincia, cantón y distrito')
    return df

inec_df = remove_column(inec_df)

In [ ]:
#Function to create a new dataframe with inec_df and oij_df when provincia, canton and distrito are equals
def join_dataframes(df1,df2):
    df = df1.join(df2, (df1.Provincia == df2.ProvinciaUnida) | (df1.Canton == df2.CantonUnido) | (df1.Distrito == df2.DistritoUnido), 'inner')
    return df

df = join_dataframes(oij_df,inec_df)
df.show(25)


In [ ]:
#Create sql table from dataframe
df.createOrReplaceTempView("df")

#Save the dataframe in a postgresql database
df.write.jdbc(url="jdbc:postgresql://localhost:5432/etl", table="etl", mode="overwrite", properties={"user": "postgres", "password": "Legolas00"})


In [ ]:
#Save the dataframe in csv file
df.write.csv(path="output.csv", sep=",", header=True,quote='"',mode="overwrite")
df.show(5)
